In [27]:
import numpy as np
import casadi as ca
from acrobotics.util import get_default_axes3d
from acrobotics.recources.robots import Kuka

robot = Kuka()

In [28]:
from casadi import cos, sin
from numpy import array

def fk_kuka2(q):
    q1, q2, q3, q4, q5, q6 = q[0], q[1], q[2], q[3], q[4], q[5]
    a1=0.18; a2=0.6; d4=0.62; d6=0.115
    T = array([
[((sin(q1)*sin(q4) + cos(q1)*cos(q4)*cos(q2 + q3))*cos(q5) - sin(q5)*sin(q2 + q3)*cos(q1))*cos(q6) + (sin(q1)*cos(q4) - sin(q4)*cos(q1)*cos(q2 + q3))*sin(q6), -((sin(q1)*sin(q4) + cos(q1)*cos(q4)*cos(q2 + q3))*cos(q5) - sin(q5)*sin(q2 + q3)*cos(q1))*sin(q6) + (sin(q1)*cos(q4) - sin(q4)*cos(q1)*cos(q2 + q3))*cos(q6), (sin(q1)*sin(q4) + cos(q1)*cos(q4)*cos(q2 + q3))*sin(q5) + sin(q2 + q3)*cos(q1)*cos(q5), a1*cos(q1) + a2*cos(q1)*cos(q2) + d4*sin(q2 + q3)*cos(q1) + d6*((sin(q1)*sin(q4) + cos(q1)*cos(q4)*cos(q2 + q3))*sin(q5) + sin(q2 + q3)*cos(q1)*cos(q5))],
[((sin(q1)*cos(q4)*cos(q2 + q3) - sin(q4)*cos(q1))*cos(q5) - sin(q1)*sin(q5)*sin(q2 + q3))*cos(q6) - (sin(q1)*sin(q4)*cos(q2 + q3) + cos(q1)*cos(q4))*sin(q6), -((sin(q1)*cos(q4)*cos(q2 + q3) - sin(q4)*cos(q1))*cos(q5) - sin(q1)*sin(q5)*sin(q2 + q3))*sin(q6) - (sin(q1)*sin(q4)*cos(q2 + q3) + cos(q1)*cos(q4))*cos(q6), (sin(q1)*cos(q4)*cos(q2 + q3) - sin(q4)*cos(q1))*sin(q5) + sin(q1)*sin(q2 + q3)*cos(q5), a1*sin(q1) + a2*sin(q1)*cos(q2) + d4*sin(q1)*sin(q2 + q3) + d6*((sin(q1)*cos(q4)*cos(q2 + q3) - sin(q4)*cos(q1))*sin(q5) + sin(q1)*sin(q2 + q3)*cos(q5))],
[                                                                (sin(q5)*cos(q2 + q3) + sin(q2 + q3)*cos(q4)*cos(q5))*cos(q6) - sin(q4)*sin(q6)*sin(q2 + q3),                                                                 -(sin(q5)*cos(q2 + q3) + sin(q2 + q3)*cos(q4)*cos(q5))*sin(q6) - sin(q4)*sin(q2 + q3)*cos(q6),                                     sin(q5)*sin(q2 + q3)*cos(q4) - cos(q5)*cos(q2 + q3),                                                                  a2*sin(q2) - d4*cos(q2 + q3) + d6*(sin(q5)*sin(q2 + q3)*cos(q4) - cos(q5)*cos(q2 + q3))],
[                                                                                                                                                           0,                                                                                                                                                             0,                                                                                       0,                                                                                                                                                        1]])
    return T

In [33]:
opti = ca.Opti()

N = 10
q = opti.variable(N, 6)

xp = np.ones(N) * 0.8
yp = np.linspace(-0.2, 0.2, N)
zp = np.ones(N) * 0.2


In [34]:
V = ca.sum1( (ca.sum2(q[:-1, :] - q[1:, :])**2 )) + 0.05* ca.sumsqr(q) #+ 1 / ca.sum1(q[:, 4]**2)

opti.minimize(V)

for i in range(N):
    Ti = fk_kuka2(q[i, :])
    opti.subject_to(xp[i] == Ti[0, 3])
    opti.subject_to(yp[i] == Ti[1, 3])
    opti.subject_to(zp[i] == Ti[2, 3])

opti.solver('ipopt');

In [35]:
%%capture
sol = opti.solve()

In [36]:
qp_sol = opti.value(q)

%matplotlib qt
fig, ax = get_default_axes3d([0, 1], [-0.5, 0.5], [0, 1])

ax.scatter(xp, yp, zp)
robot.animate_path(fig, ax, qp_sol)

In [37]:
fig, ax = get_default_axes3d([0, 1], [-0.5, 0.5], [0, 1])

ax.scatter(xp, yp, zp)
robot.plot(ax, [0, 1.5, 0, 0, 0, 0], c='k')

In [14]:
q[:, [0, 1, 2, 3, 5]].shape

(10, 5)